In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *

# project sources

## dropped/excluded

* US 50 (withdrawn)
* US 101 Marin (minimal accessibility/speed benefits, worthwhile project better analyzed in another context)
* MST reroute (apparent marginal/negative benefits, institutional barriers to relocating/adding service)

# read and aggregate

In [162]:
gpd.read_parquet('./sep9_big_blue_bus_sr2.parquet')

,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry,corridor
0,3629,1,City of Santa Monica,7.8,9.6,9.7,111.5,111.5,42.2,23,1.9,6.1,2.6,6.7,1.4,2228.8,16,"POLYGON ((141596.061 -440566.118, 141635.317 -...",bbb_sr2_am
0,3629,1,City of Santa Monica,5.6,8.3,7.7,270.4,270.4,65.3,31,3.0,5.1,4.1,5.9,1.4,2228.8,16,"POLYGON ((141596.061 -440566.118, 141635.317 -...",bbb_sr2_pm
0,3629,1,City of Santa Monica,7.7,8.8,9.1,467.7,467.7,56.5,83,6.8,6.1,8.3,6.7,1.4,2228.8,16,"POLYGON ((141596.061 -440566.118, 141635.317 -...",bbb_sr2_mid
0,3629,1,City of Santa Monica,9.3,11.5,11.4,112.5,112.5,45.1,21,2.1,4.9,2.5,5.6,1.4,2228.8,16,"POLYGON ((141596.061 -440566.118, 141635.317 -...",bbb_sr2_eve
0,3629,1,City of Santa Monica,11.1,12.6,13.0,14.7,14.7,24.0,9,0.7,6.8,0.6,6.9,1.4,2228.8,16,"POLYGON ((141596.061 -440566.118, 141635.317 -...",bbb_sr2_early_am


In [2]:
first_set = gpd.read_file('./june16_corridor_metrics_speeds.geojson')
first_set = first_set >> filter(-_.corridor.str.contains('us50')) #dropped by dist
first_set = first_set >> filter(-_.corridor.str.contains('us101_lombard')) #has hov lane
first_set['submission_round'] = 1

In [3]:
(first_set >> select(-_.geometry)).to_csv('pending_accessibility.csv')

In [4]:
second_set = gpd.read_file('./sep9_corridor_metrics_speeds.geojson')
second_set['submission_round'] = 2

In [5]:
access_results = pd.read_csv('./AccessibilityResults.csv')

In [6]:
first_set >> head(3)

,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,corridor,geometry,submission_round
2,28,28,City and County of San Francisco,9.6,11.5,11.9,1061.7,1347.9,56.4,159,14.9,5.3,18.8,6.0,2.4,3850.5,16,sr1_19th_a,"POLYGON ((-217949.193 -26918.889, -217955.214 ...",1
3,29,29,City and County of San Francisco,8.7,11.0,12.7,286.2,1347.9,70.7,180,15.0,6.0,4.0,6.1,2.4,NaN,16,sr1_19th_a,"POLYGON ((-217949.193 -26918.889, -217955.214 ...",1
4,28,28,City and County of San Francisco,10.2,11.9,13.1,506.9,956.6,56.4,159,14.9,5.3,9.0,5.6,1.4,2323.2,16,sr1_19th_b,"POLYGON ((-217636.048 -31045.226, -217636.036 ...",1


In [7]:
second_set >> head(3)

,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,corridor,geometry,submission_round
0,4-13168,4,Los Angeles County Metropolitan Transportation...,4.7,11.1,9.3,94.2,94.2,96.2,33,2.3,7.2,1.0,7.4,1.2,1982.4,16,sr2_am,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2
1,4-13168,4,Los Angeles County Metropolitan Transportation...,4.9,6.8,7.9,342.4,342.4,153.5,42,3.1,6.7,2.2,7.1,1.3,2105.6,16,sr2_pm,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2
2,4-13168,4,Los Angeles County Metropolitan Transportation...,3.4,8.8,7.8,685.4,685.4,131.2,111,6.9,8.0,5.2,8.4,1.3,2105.6,16,sr2_mid,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2


In [8]:
all_corridors = pd.concat([first_set, second_set])

In [9]:
##TODO drop organization here (access for sr2 should be both)

all_grouped = (all_corridors >> group_by(_.corridor, _.organization, _.submission_round)  # length and span cause fan-out, re-add later...
               >> summarize(sum_trips =_.n_trips.sum(), total_delay = _.total_speed_delay.max())
               >> mutate(avg_delay_minutes = _.total_delay / _.sum_trips)
)

## note: don't sum/aggregate accessibility results within a corridor

In [10]:
rider_access_joined = all_grouped >> left_join(_, access_results, on=['corridor', 'submission_round'])

In [11]:
rider_access_joined >> filter(_.corridor.str.contains('us'))

,corridor,organization,submission_round,sum_trips,total_delay,avg_delay_minutes,avg_pct_chng_access,pct_change_access_Jobs,pct_change_access_weighted_Jobs,avg_change_access_Jobs,weighted_avg_change_access_Jobs,pct_change_access_POIs,pct_change_access_weighted_POIs,avg_change_access_POIs,weighted_avg_change_access_POIs,daily_riders,pct_li_sub50k
17,us101_broadway,City of Eureka,1,72,165.4,2.297222,0.033167,0.051001,0.033613,8.485261,162.303973,0.048258,0.03272,0.165564,3.151216,600.0,0.8


# Adding ridership (added to accessibility csv)

## apportioning ridership for partial-hour bus lane on sr2

* evenly during daytime hours (0500 - 2200), match post-covid trends of less peaking
* 22,669 daily for entire corr
* 

## notes
* for SDMTS, sum ridership from routes 60, 235, 225

In [12]:
metro_4_ridership = 22669
riders_per_hour = metro_4_ridership / (22 - 5)

In [13]:
rider_estimates = (second_set >> filter(_.corridor.str.contains('sr2'))
 >> select(_.corridor, _.span_hours, _.submission_round)
 >> mutate(daily_riders = _.span_hours * riders_per_hour)
 >> select(-_.span_hours)
)
rider_estimates

,corridor,submission_round,daily_riders
0,sr2_am,2,3066.982353
1,sr2_pm,2,4133.758824
2,sr2_mid,2,9200.947059
3,sr2_eve,2,3467.023529
4,sr2_early_am,2,1333.470588


In [14]:
rider_access_joined = rider_access_joined >> left_join(_, rider_estimates, on =['corridor', 'submission_round'])

In [15]:
rider_access_joined['daily_riders'] = rider_access_joined.daily_riders_x.fillna(rider_access_joined.daily_riders_y)

In [16]:
rider_access_joined = rider_access_joined >> select(-_.daily_riders_x, -_.daily_riders_y)

# Create defined projects from disaggregated sr2

In [17]:
df = rider_access_joined >> filter(_.corridor.str.contains('sr2'), _.submission_round == 2)

In [18]:
df

,corridor,organization,submission_round,sum_trips,total_delay,avg_delay_minutes,avg_pct_chng_access,pct_change_access_Jobs,pct_change_access_weighted_Jobs,avg_change_access_Jobs,weighted_avg_change_access_Jobs,pct_change_access_POIs,pct_change_access_weighted_POIs,avg_change_access_POIs,weighted_avg_change_access_POIs,pct_li_sub50k,daily_riders
7,sr2_am,Los Angeles County Metropolitan Transportation...,2,33,94.2,2.854545,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,0.83,3066.982353
8,sr2_early_am,Los Angeles County Metropolitan Transportation...,2,14,4.1,0.292857,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,0.83,1333.470588
9,sr2_eve,Los Angeles County Metropolitan Transportation...,2,23,89.8,3.904348,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,0.83,3467.023529
10,sr2_mid,Los Angeles County Metropolitan Transportation...,2,111,685.4,6.174775,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,0.83,9200.947059
11,sr2_pm,Los Angeles County Metropolitan Transportation...,2,42,342.4,8.152381,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,0.83,4133.758824


In [19]:
def df_combined(filtered_df, corridor):
    df2 = filtered_df[['sum_trips', 'total_delay', 'daily_riders']].sum()
    df2['avg_delay_minutes'] = df2.total_delay / df2.sum_trips
    df2['corridor'] = corridor
    df2_dict = df2.to_dict()
    unchanged_series = (filtered_df >> select(-_.sum_trips, -_.total_delay, -_.daily_riders, -_.avg_delay_minutes, -_.corridor)).iloc[0,:]
    unchanged_dict = unchanged_series.to_dict()
    merged_dict = df2_dict | unchanged_dict
    merged_df = pd.DataFrame(merged_dict, index=[99])
    return merged_df

In [20]:
all_day_df = df_combined(df, 'sr2_all_day')

In [21]:
peak_only_df = df_combined(df >> filter(_.corridor.isin(['sr2_am', 'sr2_pm'])), 'sr2_peak')

In [22]:
with_sr2 = pd.concat([rider_access_joined, all_day_df, peak_only_df]).reset_index(drop=True)

In [23]:
sr2_precursors = ['sr2_am', 'sr2_pm', 'sr2_early_am', 'sr2_eve', 'sr2_mid']

In [237]:
eval_df = with_sr2 >> filter(-_.corridor.isin(sr2_precursors))

# Testing metrics

In [238]:
eval_df.columns

Index(['corridor', 'organization', 'submission_round', 'sum_trips',
       'total_delay', 'avg_delay_minutes', 'avg_pct_chng_access',
       'pct_change_access_Jobs', 'pct_change_access_weighted_Jobs',
       'avg_change_access_Jobs', 'weighted_avg_change_access_Jobs',
       'pct_change_access_POIs', 'pct_change_access_weighted_POIs',
       'avg_change_access_POIs', 'weighted_avg_change_access_POIs',
       'pct_li_sub50k', 'daily_riders'],
      dtype='object')

In [239]:
# could average both jobs and pois...

In [240]:
eval_df = (eval_df >> mutate(rider_minutes_delay = _.avg_delay_minutes * _.daily_riders)
         >> select(_.corridor, _.organization, _.submission_round,
                   _.weighted_avg_change_access_Jobs, _.weighted_avg_change_access_POIs,
                   _.avg_pct_chng_access,
                   _.pct_li_sub50k, _.daily_riders, _.rider_minutes_delay
                  )
         # >> mutate(weighted_avg_change_access_total = _.weighted_avg_change_access_Jobs)
         >> select(-_.weighted_avg_change_access_POIs)
)

In [241]:
eval_df >> filter(_.daily_riders.isna())

,corridor,organization,submission_round,weighted_avg_change_access_Jobs,avg_pct_chng_access,pct_li_sub50k,daily_riders,rider_minutes_delay
3,sr15,San Diego Metropolitan Transit System,2,NaN,NaN,NaN,NaN,NaN
14,sr66_foothill,Foothill Transit,1,NaN,NaN,NaN,NaN,NaN
15,sr82_samtrans,San Mateo County Transit District,1,NaN,NaN,NaN,NaN,NaN
16,sr82_vta,Santa Clara Valley Transportation Authority,1,NaN,NaN,NaN,NaN,NaN


In [242]:
eval_df = eval_df.dropna()

In [243]:
nineteenth = eval_df >> filter(_.corridor.str.contains('sr1_19th'))

In [244]:
nineteenth.rider_minutes_delay = nineteenth.rider_minutes_delay.sum()

/tmp/ipykernel_490/241002628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nineteenth.rider_minutes_delay = nineteenth.rider_minutes_delay.sum()


In [245]:
nineteenth.corridor = 'sr1_19th'

/tmp/ipykernel_490/2219053992.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nineteenth.corridor = 'sr1_19th'


In [246]:
eval_df = pd.concat([eval_df >> filter(-_.corridor.str.contains('sr1_19th')), nineteenth.iloc[:1,:]])

In [247]:
eval_df

,corridor,organization,submission_round,weighted_avg_change_access_Jobs,avg_pct_chng_access,pct_li_sub50k,daily_riders,rider_minutes_delay
0,i805,San Diego Metropolitan Transit System,2,1308.461745,0.015327,0.79,4233.000000,4679.816667
1,sr123,Alameda-Contra Costa Transit District,2,338.098215,0.001397,0.65,14789.000000,58885.881279
2,sr123_san_pablo,Alameda-Contra Costa Transit District,1,905.891038,0.003879,0.65,14789.000000,214334.019200
6,sr1_lincoln,City of Santa Monica,1,692.610277,0.002109,0.73,6200.000000,49918.086957
12,sr2_santa_monica_bl,Los Angeles County Metropolitan Transportation...,1,821.642091,0.002238,0.83,22669.000000,177918.999177
13,sr39_beach,Orange County Transportation Authority,1,1727.936813,0.007266,0.89,5075.000000,85674.522901
17,us101_broadway,City of Eureka,1,162.303973,0.033167,0.80,600.000000,1378.333333
18,sr2_all_day,Los Angeles County Metropolitan Transportation...,2,586.809451,0.001650,0.83,21202.182353,115604.186202
19,sr2_peak,Los Angeles County Metropolitan Transportation...,2,586.809451,0.001650,0.83,7200.741176,41917.914635
4,sr1_19th,City and County of San Francisco,1,733.304443,0.003184,0.70,12100.000000,82255.014749


In [248]:
from scipy import stats
eval_df["pctile_access"] = eval_df.avg_pct_chng_access.map(lambda x: stats.percentileofscore(eval_df.avg_pct_chng_access, x))
eval_df["pctile_rider"] = eval_df.daily_riders.map(lambda x: stats.percentileofscore(eval_df.daily_riders, x))
eval_df["pctile_rider_delay"] = eval_df.rider_minutes_delay.map(lambda x: stats.percentileofscore(eval_df.rider_minutes_delay, x))

# Draft ranking

In [249]:
eval_df['total_score'] = eval_df.pctile_access + eval_df.pctile_rider + eval_df.pctile_rider_delay
eval_df >> arrange(-_.total_score)

,corridor,organization,submission_round,weighted_avg_change_access_Jobs,avg_pct_chng_access,pct_li_sub50k,daily_riders,rider_minutes_delay,pctile_access,pctile_rider,pctile_rider_delay,total_score
2,sr123_san_pablo,Alameda-Contra Costa Transit District,1,905.891038,0.003879,0.65,14789.000000,214334.019200,70.0,75.0,100.0,245.0
12,sr2_santa_monica_bl,Los Angeles County Metropolitan Transportation...,1,821.642091,0.002238,0.83,22669.000000,177918.999177,50.0,100.0,90.0,240.0
18,sr2_all_day,Los Angeles County Metropolitan Transportation...,2,586.809451,0.001650,0.83,21202.182353,115604.186202,25.0,90.0,80.0,195.0
13,sr39_beach,Orange County Transportation Authority,1,1727.936813,0.007266,0.89,5075.000000,85674.522901,80.0,30.0,70.0,180.0
4,sr1_19th,City and County of San Francisco,1,733.304443,0.003184,0.70,12100.000000,82255.014749,60.0,60.0,60.0,180.0
1,sr123,Alameda-Contra Costa Transit District,2,338.098215,0.001397,0.65,14789.000000,58885.881279,10.0,75.0,50.0,135.0
0,i805,San Diego Metropolitan Transit System,2,1308.461745,0.015327,0.79,4233.000000,4679.816667,90.0,20.0,20.0,130.0
6,sr1_lincoln,City of Santa Monica,1,692.610277,0.002109,0.73,6200.000000,49918.086957,40.0,40.0,40.0,120.0
17,us101_broadway,City of Eureka,1,162.303973,0.033167,0.80,600.000000,1378.333333,100.0,10.0,10.0,120.0
19,sr2_peak,Los Angeles County Metropolitan Transportation...,2,586.809451,0.001650,0.83,7200.741176,41917.914635,25.0,50.0,30.0,105.0


In [250]:
# percentile rank/combine on accessibility, riders, delay
# note accessibility carry over...
# note rural projects
# drop lombard (existing HOV, operations...)
# if time, try normalization (distance from project description..., use average change access?)

In [251]:
# eval_df.plot.scatter("weighted_avg_change_access_Jobs", "daily_riders")

In [252]:
# eval_df.plot.scatter("pct_change_access_weighted_Jobs", "rider_minutes_delay")

In [253]:
eval_df.describe()

,submission_round,weighted_avg_change_access_Jobs,avg_pct_chng_access,pct_li_sub50k,daily_riders,rider_minutes_delay,pctile_access,pctile_rider,pctile_rider_delay,total_score
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000
mean,1.400000,786.386750,0.007187,0.770000,10885.792353,83256.677510,55.000000,55.000000,55.000000,165.00000
std,0.516398,454.155983,0.010074,0.082865,7427.987431,69512.678477,30.184617,30.184617,30.276504,50.71708
min,1.000000,162.303973,0.001397,0.650000,600.000000,1378.333333,10.000000,10.000000,10.000000,105.00000
25%,1.000000,586.809451,0.001765,0.707500,5356.250000,43917.957716,28.750000,32.500000,32.500000,122.50000
50%,1.000000,712.957360,0.002711,0.795000,9650.370588,70570.448014,55.000000,55.000000,55.000000,157.50000
75%,2.000000,884.828801,0.006419,0.830000,14789.000000,108121.770376,77.500000,75.000000,77.500000,191.25000
max,2.000000,1727.936813,0.033167,0.890000,22669.000000,214334.019200,100.000000,100.000000,100.000000,245.00000


In [254]:
first_filtered = first_set >> distinct(_.corridor, _.length_miles, _.geometry)

In [255]:
second_filtered = second_set >> distinct(_.corridor, _.length_miles, _.geometry) >> filter(-_.corridor.isin(['sr2_am', 'sr2_mid', 'sr2_eve']))

In [256]:
# second_filtered.explore()

In [257]:
second_filtered = second_filtered.dissolve('corridor').reset_index()

In [258]:
second_filtered

,corridor,geometry,length_miles
0,i805,"POLYGON ((272461.912 -585285.830, 272364.596 -...",1.2
1,sr123,"POLYGON ((-200526.017 -17118.327, -200519.214 ...",1.1
2,sr15,"POLYGON ((270968.467 -582950.917, 270992.769 -...",1.0
3,sr2_early_am,"POLYGON ((140814.209 -441265.975, 140892.548 -...",1.3
4,sr2_pm,"POLYGON ((140814.209 -441265.975, 140892.548 -...",1.3


In [259]:
second_filtered.corridor = second_filtered.corridor.str.replace('sr2_early_am', 'sr2_all_day')
second_filtered.corridor = second_filtered.corridor.str.replace('sr2_pm', 'sr2_peak')

In [260]:
first_filtered.corridor = first_filtered.corridor.str.replace('sr1_19th_a', 'sr1_19th')
first_filtered.corridor = first_filtered.corridor.str.replace('sr1_19th_b', 'sr1_19th')

In [261]:
first_filtered = first_filtered.dissolve('corridor').reset_index()

In [262]:
both_filtered = pd.concat([first_filtered, second_filtered])

In [263]:
from calitp_data_analysis import geography_utils

In [264]:
with_geom = eval_df >> left_join(_, both_filtered, on = 'corridor')
with_geom = gpd.GeoDataFrame(with_geom, crs=geography_utils.CA_NAD83Albers)

In [265]:
# with_geom.explore()

# Normalized draft ranking

In [266]:
# with_geom = with_geom >> select(-_.pctile_access, -_.pctile_rider, -_.pctile_rider_delay)

In [267]:
with_geom = with_geom >> mutate(access_mi = _.weighted_avg_change_access_Jobs / _.length_miles,
                    riders_mi = _.daily_riders / _.length_miles,
                    rider_delay_mi = _.rider_minutes_delay / _.length_miles
                   )

In [268]:
with_geom["pctile_access_norm"] = with_geom.access_mi.map(lambda x: stats.percentileofscore(with_geom.access_mi, x))
with_geom["pctile_rider_norm"] = with_geom.riders_mi.map(lambda x: stats.percentileofscore(with_geom.riders_mi, x))
with_geom["pctile_rider_delay_norm"] = with_geom.rider_delay_mi.map(lambda x: stats.percentileofscore(with_geom.rider_delay_mi, x))

with_geom['total_score_norm'] = with_geom.pctile_access_norm + with_geom.pctile_rider_norm + with_geom.pctile_rider_delay_norm
# with_geom >> arrange(-_.total_score_norm)

In [269]:
with_geom.columns

Index(['corridor', 'organization', 'submission_round',
       'weighted_avg_change_access_Jobs', 'avg_pct_chng_access',
       'pct_li_sub50k', 'daily_riders', 'rider_minutes_delay', 'pctile_access',
       'pctile_rider', 'pctile_rider_delay', 'total_score', 'geometry',
       'length_miles', 'access_mi', 'riders_mi', 'rider_delay_mi',
       'pctile_access_norm', 'pctile_rider_norm', 'pctile_rider_delay_norm',
       'total_score_norm'],
      dtype='object')

In [270]:
(with_geom >> select(-_.geometry)).to_csv('detailed_scores.csv')

In [271]:
summary = with_geom >> select(_.corridor, _.organization, _.submission_round,
                   _.pct_li_sub50k, _.length_miles, _.total_score,
                   _.total_score_norm
                   )

In [231]:
## TODO share csv with subscores -> Henry
summary >> arrange(-_.total_score)

,corridor,organization,submission_round,pct_li_sub50k,length_miles,total_score,total_score_norm
2,sr123_san_pablo,Alameda-Contra Costa Transit District,1,0.65,6.1,245.0,130.0
4,sr2_santa_monica_bl,Los Angeles County Metropolitan Transportation...,1,0.83,2.0,240.0,250.0
7,sr2_all_day,Los Angeles County Metropolitan Transportation...,2,0.83,1.3,195.0,275.0
5,sr39_beach,Orange County Transportation Authority,1,0.89,12.7,180.0,50.0
9,sr1_19th,City and County of San Francisco,1,0.70,2.4,180.0,170.0
1,sr123,Alameda-Contra Costa Transit District,2,0.65,1.1,135.0,230.0
0,i805,San Diego Metropolitan Transit System,2,0.79,1.2,130.0,170.0
3,sr1_lincoln,City of Santa Monica,1,0.73,4.1,120.0,100.0
6,us101_broadway,City of Eureka,1,0.80,0.8,120.0,70.0
8,sr2_peak,Los Angeles County Metropolitan Transportation...,2,0.83,1.3,105.0,205.0


In [232]:
summary >> arrange(-_.total_score_norm)

,corridor,organization,submission_round,pct_li_sub50k,length_miles,total_score,total_score_norm
7,sr2_all_day,Los Angeles County Metropolitan Transportation...,2,0.83,1.3,195.0,275.0
4,sr2_santa_monica_bl,Los Angeles County Metropolitan Transportation...,1,0.83,2.0,240.0,250.0
1,sr123,Alameda-Contra Costa Transit District,2,0.65,1.1,135.0,230.0
8,sr2_peak,Los Angeles County Metropolitan Transportation...,2,0.83,1.3,105.0,205.0
0,i805,San Diego Metropolitan Transit System,2,0.79,1.2,130.0,170.0
9,sr1_19th,City and County of San Francisco,1,0.70,2.4,180.0,170.0
2,sr123_san_pablo,Alameda-Contra Costa Transit District,1,0.65,6.1,245.0,130.0
3,sr1_lincoln,City of Santa Monica,1,0.73,4.1,120.0,100.0
6,us101_broadway,City of Eureka,1,0.80,0.8,120.0,70.0
5,sr39_beach,Orange County Transportation Authority,1,0.89,12.7,180.0,50.0
